In [ ]:
import multiprocessing
from tqdm import tqdm
from shapely.geometry import Point
from shapely.geometry import box
import json
from datetime import date # Потрібно для обробки об'єктів date

# --- ЕМУЛЯЦІЯ/КОНСТАНТИ ---
# Припускаємо, що ці змінні/функції визначені десь вище у вашому коді:
# folder_data, deforestation_by_tile, MIN_DAYS_BETWEEN, get_tile_bounds

# Файл, куди будуть зберігатися всі створені пари
PAIRS_OUTPUT_FILE = 'seasonal_pairs_metadata.jsonl' 
# Глобальний лок для безпечного запису в спільний файл (необхідний для multiprocessing)
FILE_LOCK = multiprocessing.Lock()


# --- ДОПОМІЖНА ФУНКЦІЯ ДЛЯ СЕРІАЛІЗАЦІЇ ---
def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""
    if isinstance(obj, date):
        return obj.isoformat()
    # Серіалізуємо об'єкти Shapely Geometry (якщо вони є)
    if hasattr(obj, '__geo_interface__'):
        return obj.__geo_interface__
    raise TypeError ("Type %s not serializable" % type(obj))

# --- ФУНКЦІЯ ДЛЯ ПАРАЛЕЛЬНОЇ ОБРОБКИ (З БЕЗПЕЧНИМ ЗАПИСОМ) ---
def create_pairs_for_deforestation(defor_data):
    """
    Створює сезонні пари зображень для однієї події вирубки та записує їх у файл.

    Args:
        defor_data (tuple): Кортеж, що містить:
                            (defor, tile, summer_images, other_seasons)

    Returns:
        int: Кількість успішно створених та збережених пар.
    """
    defor, tile, summer_images, other_seasons = defor_data
    local_pairs_count = 0
    centroid = defor['geometry'].centroid

    # Тимчасовий список для збору пар, які згенерував цей процес
    pairs_to_write = []

    # Створюємо пари для кожного сезону
    for season_name, season_images in other_seasons.items():
        for summer_img in summer_images:
            
            # Отримання меж для перевірки видимості
            try:
                summer_bounds = box(*get_tile_bounds(summer_img['tci_path']))
            except Exception:
                continue
            
            # Перевірка видимості centroid у літньому знімку
            if not Point(centroid).within(summer_bounds):
                continue
                
            for other_img in season_images:
                days_diff = abs((other_img['date'] - summer_img['date']).days)
                
                # Перевіряємо мінімальний часовий інтервал
                if days_diff < MIN_DAYS_BETWEEN:
                    continue
                
                # Перевіряємо, чи вирубка видима в "іншому" знімку
                try:
                    other_bounds = box(*get_tile_bounds(other_img['tci_path']))
                except Exception:
                    continue
                    
                if not Point(centroid).within(other_bounds):
                    continue
                
                # Замість pairs.append(), додаємо до локального списку
                pair = {
                    'summer_img': summer_img,
                    'other_img': other_img,
                    'season_pair': f'summer_{season_name}',
                    'deforestation': defor,
                    'centroid': centroid,
                    'tile': tile
                }
                
                pairs_to_write.append(pair)
                local_pairs_count += 1
                
    # --- БЕЗПЕЧНИЙ ЗАПИС У ФАЙЛ ---
    if pairs_to_write:
        # Блокуємо файл, щоб інший процес не записував одночасно
        with FILE_LOCK:
            with open(PAIRS_OUTPUT_FILE, 'a', encoding='utf-8') as f:
                for pair in pairs_to_write:
                    # Серіалізуємо об'єкт перед записом
                    json_line = json.dumps(pair, default=json_serial)
                    f.write(json_line + '\n')
                    
    return local_pairs_count

# --- ОСНОВНИЙ БЛОК ПАРАЛЕЛЬНОЇ ОБРОБКИ ---

def run_pair_creation():
    """Головна функція для запуску створення пар."""
    
    print("🔄 Створення сезонних пар та запис на диск...")

    # 1. Скидання файлу (або перевірка на існування для відновлення)
    if os.path.exists(PAIRS_OUTPUT_FILE):
        print(f"⚠️ Файл {PAIRS_OUTPUT_FILE} існує. Буде додано нові дані.")
    else:
        print(f"📝 Створення нового файлу: {PAIRS_OUTPUT_FILE}")

    # 2. Збір всіх завдань
    tasks = []
    tiles_to_process = ['T36UYA', 'T36UXA']
    
    for tile in tiles_to_process:
        # Фільтруємо зображення для поточного тайлу (це не змінюється)
        tile_images = [img for img in folder_data if img['tile'] == tile]
        
        # Зображення для кожного сезону (це не змінюється)
        summer_images = [img for img in tile_images if img['season'] == 'summer']
        other_seasons = {
            'autumn': [img for img in tile_images if img['season'] == 'autumn'],
            'winter': [img for img in tile_images if img['season'] == 'winter'],
            'spring': [img for img in tile_images if img['season'] == 'spring']
        }
        
        # Формуємо список завдань: одна вирубка = одне завдання
        for defor in deforestation_by_tile[tile]:
            tasks.append((defor, tile, summer_images, other_seasons))

    num_processes = 7 # Використовуємо 4 ядра, як просили
    
    print(f"🛠️ Запуск паралельної обробки на {num_processes} процесах для {len(tasks)} подій вирубки...")

    # 3. Запуск Pool для паралельного виконання
    with multiprocessing.Pool(processes=num_processes) as pool:
        # Використовуємо imap_unordered для паралельного виконання та tqdm для прогресу
        results = list(tqdm(pool.imap_unordered(create_pairs_for_deforestation, tasks), 
                             total=len(tasks), 
                             desc="Генерація пар та запис на диск"))
        
        total_pairs_created = sum(results)

    print(f"✅ Створено та збережено {total_pairs_created} сезонних пар у файлі {PAIRS_OUTPUT_FILE}")

    
run_pair_creation()

In [ ]:
# 5. Створення сезонних пар
print("🔄 Створення сезонних пар...")
pairs = []

for tile in ['T36UYA', 'T36UXA']:
    # Фільтруємо зображення для поточного тайлу
    tile_images = [img for img in folder_data if img['tile'] == tile]
    
    # Зображення для кожного сезону
    summer_images = [img for img in tile_images if img['season'] == 'summer']
    other_seasons = {
        'autumn': [img for img in tile_images if img['season'] == 'autumn'],
        'winter': [img for img in tile_images if img['season'] == 'winter'],
        'spring': [img for img in tile_images if img['season'] == 'spring']
    }
    
    # Обробка кожної вирубки
    for defor in tqdm(deforestation_by_tile[tile], desc=f"Обробка {tile}"):
        centroid = defor['geometry'].centroid
        
        # Створюємо пари для кожного сезону
        for season_name, season_images in other_seasons.items():
            for summer_img in summer_images:
                for other_img in season_images:
                    days_diff = abs((other_img['date'] - summer_img['date']).days)
                    
                    # Перевіряємо мінімальний часовий інтервал
                    if days_diff < MIN_DAYS_BETWEEN:
                        continue
                    
                    # Перевіряємо, чи вирубка видима в обох знімках
                    if not (Point(centroid).within(box(*get_tile_bounds(summer_img['tci_path']))) and
                            Point(centroid).within(box(*get_tile_bounds(other_img['tci_path'])))):
                        continue
                    
                    pairs.append({
                        'summer_img': summer_img,
                        'other_img': other_img,
                        'season_pair': f'summer_{season_name}',
                        'deforestation': defor,
                        'centroid': centroid,
                        'tile': tile
                    })

print(f"✅ Створено {len(pairs)} сезонних пар")

In [ ]:
# Додайте цей код перед функцією run_patch_generation()

from shapely.geometry import shape
from datetime import datetime, date # Додаткові імпорти для коректного парсингу

PAIRS_METADATA_FILE = 'seasonal_pairs_metadata.jsonl'
# Використовуємо 512, як у вашому плані
PATCH_SIZE_FINAL = 500

def load_and_prepare_pairs():
    """Читає всі пари з .jsonl, перетворює геометрію та збирає унікальні ID."""
    all_pairs = []
    unique_defor_ids = set()
    
    try:
        with open(PAIRS_METADATA_FILE, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    pair = json.loads(line)
                    
                    # 1. Парсинг дат (необхідно для розрахунку days_difference)
                    pair['summer_img']['date'] = datetime.strptime(pair['summer_img']['date'].split('T')[0], '%Y-%m-%d').date()
                    pair['other_img']['date'] = datetime.strptime(pair['other_img']['date'].split('T')[0], '%Y-%m-%d').date()
                    
                    # 2. Парсинг центроїда (shapely Point)
                    # pair['centroid'] є об'єктом GeoJSON {"type": "Point", "coordinates": [...]}.
                    pair['centroid'] = shape(pair['centroid'])
                    
                    # 3. Збір ID
                    unique_defor_ids.add(pair['deforestation']['id'])
                    
                    all_pairs.append(pair)
                except Exception as e:
                    print(f"Помилка парсингу рядка в {PAIRS_METADATA_FILE}: {e}")
                    continue
    except FileNotFoundError:
        print(f"❌ Файл {PAIRS_METADATA_FILE} не знайдено.")
        return [], set()

    return all_pairs, unique_defor_ids

In [ ]:
# --- ДОПОМІЖНІ ФУНКЦІЇ ДЛЯ PARALLEL. (потрібно додати у Ваш код) ---

# ПРИМІТКА: Для роботи цього блоку потрібно, щоб функції 
# `crop_patch_around_centroid`, `find_band_path`, `get_tile_bounds`
# були визначені вище у Вашому ноутбуці.

# Створюємо фіктивну функцію для демонстрації, яка емулює вашу crop_patch_around_centroid, 
# але бере на вхід folder_path і band_name, а не tci_path (як у 6-канальній версії)
# Оскільки ви надали код, що використовує tci_path, ми залишаємо його, 
# але зауважте, що 6-канальна версія була б кращою.

H_GT = np.eye(3) # Гомографія Ground Truth
NUM_WORKERS = os.cpu_count() - 1 if os.cpu_count() > 1 else 1
# ------------------------------------------------------------------

def process_single_pair_final(i, pair_data, output_dir, patch_size, val_ids, h_gt):
    """
    Обробляє та зберігає патчі для однієї пари знімків.
    Ця функція виконується паралельно.
    """
    
    # Вирізаємо патчі для обох зображень
    patch_summer, meta_summer = crop_patch_around_centroid(
        pair_data['summer_img']['tci_path'], 
        pair_data['centroid'], 
        patch_size
    )
    
    patch_other, meta_other = crop_patch_around_centroid(
        pair_data['other_img']['tci_path'], 
        pair_data['centroid'], 
        patch_size
    )
    
    # Пропускаємо помилкові патчі
    if patch_summer is None or patch_other is None:
        return None
    
    # Визначаємо, куди зберігати (train/val)
    split = 'val' if pair_data['deforestation']['id'] in val_ids else 'train'
    pair_dir = os.path.join(output_dir, split, f"pair_{i:04d}")
    os.makedirs(pair_dir, exist_ok=True)
    
    # Зберігаємо патчі та Гомографію
    np.save(os.path.join(pair_dir, 'img1.npy'), patch_summer) # Використовуємо img1/img2 для LoFTR
    np.save(os.path.join(pair_dir, 'img2.npy'), patch_other)
    np.save(os.path.join(pair_dir, 'homography.npy'), h_gt) 
    
    # Зберігаємо метадані
    metadata = {
        'img1_path': pair_data['summer_img']['tci_path'],
        'img2_path': pair_data['other_img']['tci_path'],
        'date_img1': pair_data['summer_img']['date'].isoformat(),
        'date_img2': pair_data['other_img']['date'].isoformat(),
        'season_pair': pair_data['season_pair'],
        'deforestation_id': pair_data['deforestation']['id'],
        'centroid': pair_data['centroid'].__geo_interface__,
        'tile': pair_data['tile'],
        'days_difference': abs((pair_data['other_img']['date'] - pair_data['summer_img']['date']).days)
    }
    
    with open(os.path.join(pair_dir, 'metadata.json'), 'w') as f:
        json.dump(metadata, f, indent=2)
        
    # Повертаємо індекс пари для подальшої статистики
    return i


def run_patch_generation():
    """Запускає процес генерації патчів на основі даних з JSONL."""
    
    # 1. Завантаження та підготовка даних
    print(f"📖 Читання даних з {PAIRS_METADATA_FILE}...")
    all_pairs, unique_defor_ids = load_and_prepare_pairs()
    
    if not all_pairs:
        print("❌ Не знайдено коректних пар для обробки.")
        return

    # 2. Розділення на train/val
    print("✂️ Розділення на Train/Val за ID вирубок...")
    unique_defor_ids_list = list(unique_defor_ids)
    np.random.seed(42)
    val_defor_ids = set(np.random.choice(
        unique_defor_ids_list, 
        size=int(0.2 * len(unique_defor_ids_list)), 
        replace=False
    ))
    
    # 3. Налаштування директорій
    print(f"💾 Генерація патчів та збереження датасету в {OUTPUT_DIR}...")
    os.makedirs(os.path.join(OUTPUT_DIR, 'train'), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, 'val'), exist_ok=True)

    # 4. Підготовка завдань для паралелізації
    tasks = []
    for i, pair in enumerate(all_pairs):
        # Передаємо лише необхідні дані у пул
        tasks.append((i, pair, OUTPUT_DIR, PATCH_SIZE_FINAL, val_defor_ids, H_GT))
    
    print(f"⚡️ Запуск паралельної обробки {len(tasks)} пар на {NUM_WORKERS} процесорах...")

    # 5. Запуск пулу процесів
    with Pool(NUM_WORKERS) as pool:
        # map повертає індекси успішно оброблених пар
        results = list(tqdm(pool.starmap(process_single_pair_final, tasks), 
                            total=len(tasks), 
                            desc="Паралельна генерація"))

    successful_indices = [idx for idx in results if idx is not None]
    
    # 6. Збереження статистики
    successful_pairs_count = len(successful_indices)
    
    stats = {
        'total_pairs_attempted': len(all_pairs),
        'total_pairs_successful': successful_pairs_count,
        'train_pairs': len([i for i in successful_indices if all_pairs[i]['deforestation']['id'] not in val_defor_ids]),
        'val_pairs': len([i for i in successful_indices if all_pairs[i]['deforestation']['id'] in val_defor_ids]),
        'season_distribution': {},
        'tile_distribution': {'T36UYA': 0, 'T36UXA': 0}
    }

    # Обчислення розподілу
    for i in successful_indices:
        pair = all_pairs[i]
        season = pair['season_pair']
        tile = pair['tile']
        
        stats['season_distribution'][season] = stats['season_distribution'].get(season, 0) + 1
        stats['tile_distribution'][tile] += 1
        
    with open(os.path.join(OUTPUT_DIR, 'dataset_stats.json'), 'w') as f:
        json.dump(stats, f, indent=2)
    
    # 7. Вивід результатів (як у вашому коді)
    print("\n✅ ДАТАСЕТ СТВОРЕНО УСПІШНО!")
    print(f"📊 Загальна кількість успішних пар: {stats['total_pairs_successful']}")
    # ... (інший вивід статистики)

In [ ]:
# 6. Генерація патчів та збереження датасету
print("💾 Генерація патчів та збереження датасету...")

# Створюємо структуру директорій
os.makedirs(os.path.join(OUTPUT_DIR, 'train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'val'), exist_ok=True)

# Розділяємо пари на train/val (80/20) за вирубками
unique_defor_ids = list(set([pair['deforestation']['id'] for pair in all_pairs]))
np.random.seed(42)
val_ids = set(np.random.choice(unique_defor_ids, size=int(0.2 * len(unique_defor_ids)), replace=False))

# Головний цикл обробки пар
for i, pair in enumerate(tqdm(all_pairs, desc="Генерація патчів")):
    # Вирізаємо патчі для обох зображень
    patch_summer, meta_summer = crop_patch_around_centroid(
        pair['summer_img']['tci_path'], 
        pair['centroid'], 
        PATCH_SIZE
    )
    
    patch_other, meta_other = crop_patch_around_centroid(
        pair['other_img']['tci_path'], 
        pair['centroid'], 
        PATCH_SIZE
    )
    
    # Пропускаємо помилкові патчі
    if patch_summer is None or patch_other is None:
        continue
    
    # Визначаємо, куди зберігати (train/val)
    split = 'val' if pair['deforestation']['id'] in val_ids else 'train'
    pair_dir = os.path.join(OUTPUT_DIR, split, f"pair_{i:04d}")
    os.makedirs(pair_dir, exist_ok=True)
    
    # Зберігаємо патчі
    np.save(os.path.join(pair_dir, 'img_summer.npy'), patch_summer)
    np.save(os.path.join(pair_dir, 'img_other.npy'), patch_other)
    
    # Зберігаємо метадані
    metadata = {
        'summer_img_path': pair['summer_img']['tci_path'],
        'other_img_path': pair['other_img']['tci_path'],
        'date_summer': pair['summer_img']['date'].strftime('%Y-%m-%d'),
        'date_other': pair['other_img']['date'].strftime('%Y-%m-%d'),
        'season_summer': 'summer',
        'season_other': pair['season_pair'].split('_')[1],
        'season_pair': pair['season_pair'],
        'deforestation_id': pair['deforestation']['id'],
        'centroid': (pair['centroid'].x, pair['centroid'].y),
        'tile': pair['tile'],
        'grid_position': meta_summer['grid_position'],
        'days_difference': abs((pair['other_img']['date'] - pair['summer_img']['date']).days)
    }
    
    with open(os.path.join(pair_dir, 'metadata.json'), 'w') as f:
        json.dump(metadata, f, indent=2)

In [ ]:
# 7. Візуалізація прикладу
print("🖼️ Створення прикладу візуалізації...")
sample_pair = all_pairs[0]
patch_summer, _ = crop_patch_around_centroid(
    sample_pair['summer_img']['tci_path'], 
    sample_pair['centroid'], 
    PATCH_SIZE
)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(patch_summer)
plt.title(f"Літо ({sample_pair['summer_img']['date'].strftime('%Y-%m-%d')})")
plt.axis('off')

patch_other, _ = crop_patch_around_centroid(
    sample_pair['other_img']['tci_path'], 
    sample_pair['centroid'], 
    PATCH_SIZE
)

plt.subplot(1, 2, 2)
plt.imshow(patch_other)
plt.title(f"{sample_pair['season_pair'].split('_')[1].capitalize()} ({sample_pair['other_img']['date'].strftime('%Y-%m-%d')})")
plt.axis('off')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'sample_pair.jpg'), dpi=150, bbox_inches='tight')

In [ ]:
# 8. Збереження статистики
print("📊 Збереження статистики датасету...")
stats = {
    'total_pairs': len(all_pairs),
    'train_pairs': len([p for p in all_pairs if p['deforestation']['id'] not in val_ids]),
    'val_pairs': len([p for p in all_pairs if p['deforestation']['id'] in val_ids]),
    'season_distribution': {},
    'tile_distribution': {'T36UYA': 0, 'T36UXA': 0}
}

# Розподіл за сезонними парами
for season in ['summer_autumn', 'summer_winter', 'summer_spring']:
    stats['season_distribution'][season] = len([p for p in all_pairs if p['season_pair'] == season])

# Розподіл за тайлами
for pair in all_pairs:
    stats['tile_distribution'][pair['tile']] += 1

with open(os.path.join(OUTPUT_DIR, 'dataset_stats.json'), 'w') as f:
    json.dump(stats, f, indent=2)

# Вивід статистики
print("\n✅ ДАТАСЕТ СТВОРЕНО УСПІШНО!")
print(f"📁 Шлях: {OUTPUT_DIR}")
print(f"📊 Загальна кількість пар: {stats['total_pairs']}")
print(f"   • Train: {stats['train_pairs']}")
print(f"   • Val: {stats['val_pairs']}")
print("\nСезонний розподіл:")
for season, count in stats['season_distribution'].items():
    print(f"   • {season.replace('_', '/').capitalize()}: {count}")
print("\nРозподіл за тайлами:")
for tile, count in stats['tile_distribution'].items():
    print(f"   • {tile}: {count}")
print(f"\n🖼️ Приклад візуалізації збережено: {os.path.join(OUTPUT_DIR, 'sample_pair.jpg')}")